# Testing

In [1]:
import pandas as pd
from fine_tune_inference import FineTuneInference

In [2]:
def clean_messages(message: list):
    input_message = []
    gt_answer = None
    for m in message:
        if m['role'] == 'user' or m['role'] == 'system':
            input_message.append(m)
        elif m['role'] == 'assistant':
            gt_answer = m['content']
    return input_message, gt_answer

In [3]:
val_df = pd.read_json('Sample-Size-LLM/val.jsonl', lines=True, orient='records')

In [4]:
fti = FineTuneInference(adapter_dir='Sample-Size-LLM/sample-size-sft-lora')

Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

ValueError: Can't find 'adapter_config.json' at 'Sample-Size-LLM/sample-size-sft-lora'

In [6]:
val_df.messages[1]

[{'role': 'system',
  'content': 'You are an expert data analyst. Given summarized data from a tabular dataset, you will be asked to perform various statistical analyses. Return ONLY the final sample size as an integer.'},
 {'role': 'user',
  'content': 'Population size: 107\nConfidence level: 0.9\nTolerable error rate: 0.1\nAssumed probability of success: 0.5\nRounding: ceil\nUsed FPC: True\n\nReturn ONLY the final sample size.'},
 {'role': 'assistant', 'content': '42'}]

In [8]:
test_messages = []
actual = []
for val in val_df.messages:
    imess, gt = clean_messages(val)
    test_messages.append(imess)
    actual.append(gt)

In [9]:
predictions = [fti.predict(x) for x in test_messages]

In [10]:
pd.DataFrame({
    'actual': actual,
    'predictions': predictions
})

,actual,predictions
0,49,49
1,42,42
2,36,36
3,40,40
4,35,34
5,49,48
6,47,48
7,47,47
8,35,34
9,33,32


In [ ]:
# 2. Perform a variance analysis on Q2 and Q3 data (columns H and I).
# - Calculate quarter-on-quarter variance and capture the result in column J.

# For each csv:
# - Rename quarter columns
# - Filter to Q columns and convert to markdown

In [5]:
task2_df = pd.read_csv('audit_synth/ex_00000/population.csv')

In [6]:
task2_df.head()

,Metric,Region,Owner,Unit,Q2_2024,Q3_2024,SourceType
0,SAR Filing Timeliness,NaN,AFC Ops,count,3249,3320,Reported
1,KYC Exceptions Rate,LATAM,Risk Analytics,score,71,87,Calculated
2,Transaction Monitoring Alerts,EMEA,Compliance,count,4237,4402,Reported
3,PEP Screening Hits,LATAM,AFC Ops,rate,94,98,Calculated
4,EDD Completion Rate,EMEA,AFC Ops,count,1163,524,Calculated


In [7]:
# qdf = task2_df[['Q2_2024', 'Q3_2024']].to_markdown(index=False)
qdf = task2_df[['Q2_2024', 'Q3_2024']]

In [12]:
qdf['Variance'] = [round(((a - b)/b)*100, 0) if b !=0 else 0 for a, b in zip(qdf['Q2_2024'], qdf['Q3_2024'])]


In [14]:
mdqv = qdf['Variance'].to_markdown(index=False)
print(mdqv)

|   Variance |
|-----------:|
|         -2 |
|        -18 |
|         -4 |
|         -4 |
|        122 |
|        -32 |
|         -6 |
|        -40 |
|        -14 |
|         -4 |
|         -4 |
|         -7 |
|        -10 |
|         -7 |
|         13 |
|         22 |
|         25 |
|         -7 |
|        166 |
|       -100 |
|         -5 |
|        100 |
|         10 |
|         43 |
|         23 |
|         42 |
|        -54 |
|        -69 |
|       -100 |
|          2 |
|        -34 |
|        -50 |
|         16 |
|         -8 |
|         19 |
|         18 |
|         10 |
|          0 |
|        -60 |
|         -6 |
|          3 |
|         19 |
|         75 |
|         13 |
|          8 |
|         -7 |
|        -25 |
|          2 |
|        -10 |
|          9 |
|          7 |
|         12 |
|          3 |
|         -2 |
|          0 |
|         -2 |
|          7 |
|          1 |
|         27 |
|        -10 |
|         -3 |
|        -27 |
|          5 |
|         -8 |
|        2

In [15]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct", use_fast=True)


In [16]:
len(tokenizer.encode(mdqv))

1173

In [8]:

new_task_message = [
    {
        "role": "system",
        "content": "You are an expert data analyst. Given summarized data from a tabular dataset, you will be asked to perform various statistical analyses. Return ONLY the quarter-on-quarter variance as a floating point number."},
    {
        "role": "user",
        "content": f"Quarterly Data: {qdf}\n\nReturn ONLY the quarter-on-quarter variance."}
]

In [9]:
fti.predict(new_task_message, 1000)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


'1.51'

In [ ]:
# There is data in the attached file, <FILE>. I need to use the data to be able to accomplish <TASK> where the end result is <GT EXAMPLE>. Following the format of the data in the file, generate 400 synthetic examples of both the data in the file and a corresponding ground truth result.

In [33]:
sample_size_calculation1 = {
  "confidence_level": 0.9,
  "z_value": 1.64485,
  "tolerable_error": 0.1,
  "assumed_p": 0.5,
  "n0_unbounded": 67.63828806249998,
  "population_size": 203,
  "finite_population_corrected_n": 50.92219126352285,
  "final_sample_size": 51,
  "used_fpc": True,
  "rounding": "ceil",
  "formula_notes": {
    "n0": "(Z^2 * p * (1-p)) / E^2",
    "fpc": "(N * n0) / (N + n0 - 1)"
  }
}

In [35]:
population_size = sample_size_calculation1['population_size']
confidence_level = sample_size_calculation1['confidence_level']
tolerable_error = sample_size_calculation1['tolerable_error']
assumed_p = sample_size_calculation1['assumed_p']
ground_truth = sample_size_calculation1['final_sample_size']

In [36]:
sample_training_data = {
    'prompt': [
        {'content': "You are an expert data analyst. Given a tabular set of data, you will be asked to perform various statistical analyses.",
         'role': 'system'},
        {'content': f"There is a data set that has a population size of {population_size}. Calculate the required sample size for audit testing using the following statistical metrics:\n\n- Confidence level: {confidence_level}\n- Tolerable error rate: {tolerable_error}\n- Assumed probability of success: {assumed_p}\n\nReturn ONLY the final sample size.",
         'role': 'user'}
    ],
    'ground_truth': f"Final sample size: {ground_truth}"
}